In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import splitfolders
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,InputLayer
from tensorflow.keras.losses import BinaryCrossentropy

2025-04-13 14:33:09.331996: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-13 14:33:09.645341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744554789.762575    2259 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744554789.800496    2259 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744554789.986023    2259 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [5]:
splitfolders.ratio('faces','dataset',seed = 42,ratio=(.8,.1,.1))


Copying files: 27167 files [10:09, 44.58 files/s]


In [3]:
train_data = 'dataset/train'
test_data = 'dataset/test'
val_data ='dataset/val'

In [4]:
img_size = 180

In [5]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_data,
    image_size=(img_size,img_size),
    batch_size=32,
    shuffle=True,
    validation_split=False
)

Found 21733 files belonging to 2 classes.


I0000 00:00:1744554824.504403    2259 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [6]:
data_category = train_dataset.class_names

In [7]:
data_category

['man', 'woman']

In [8]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_data,
    image_size=(img_size,img_size),
    batch_size=32,
    shuffle=False,
    validation_split=False
)

Found 2719 files belonging to 2 classes.


In [9]:
val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_data,
    image_size=(img_size,img_size),
    batch_size=32,
    shuffle=False,
    validation_split=False
)

Found 2715 files belonging to 2 classes.


In [10]:
train_dataset = train_dataset.shuffle(120,reshuffle_each_iteration=True).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(120,reshuffle_each_iteration=True).prefetch(tf.data.AUTOTUNE)

In [11]:
model = tf.keras.Sequential([
    layers.Rescaling(1./255),
    Conv2D(16 , (3,3) , padding = 'same' , activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),

    Conv2D(32 , (3,3) , padding = 'same' , activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),

    Conv2D(64 , (3,3) , padding = 'same' , activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),

    layers.Flatten(),

    Dense(256, activation='relu'),
    Dense(128 , activation = 'relu'),
    
    layers.Dropout(0.2),
    
    Dense(1 , activation = 'sigmoid')
])

In [12]:
model.compile(optimizer = 'adam' , loss = BinaryCrossentropy() , metrics = ['accuracy'])

In [13]:
model.fit(train_dataset , validation_data = val_dataset , epochs = 20 , verbose = 1 , callbacks = [ tf.keras.callbacks.EarlyStopping(
    monitor = 'val_accuracy',
    patience = 3,
    restore_best_weights = True
),
    tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',  
    factor=0.6,  
    patience=4, 
    min_lr=1e-6  
)])

Epoch 1/20


I0000 00:00:1744554842.909878    2359 service.cc:152] XLA service 0x7f15c401e990 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744554842.910882    2359 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-04-13 14:34:03.127658: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744554843.548504    2359 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-13 14:34:05.285120: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1049', 32 bytes spill stores, 32 bytes spill loads

2025-04-13 14:34:05.682449: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusio

  1/680 ━━━━━━━━━━━━━━━━━━━━ 3:16:22 17s/step - accuracy: 0.6875 - loss: 0.6800

I0000 00:00:1744554852.467926    2359 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


680/680 ━━━━━━━━━━━━━━━━━━━━ 61s 64ms/step - accuracy: 0.7467 - loss: 0.5098 - val_accuracy: 0.8158 - val_loss: 0.3927 - learning_rate: 0.0010
Epoch 2/20
680/680 ━━━━━━━━━━━━━━━━━━━━ 25s 33ms/step - accuracy: 0.8745 - loss: 0.2921 - val_accuracy: 0.8980 - val_loss: 0.2521 - learning_rate: 0.0010
Epoch 3/20
679/680 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9168 - loss: 0.2077 

...

680/680 ━━━━━━━━━━━━━━━━━━━━ 21s 29ms/step - accuracy: 0.9168 - loss: 0.2077 - val_accuracy: 0.9215 - val_loss: 0.2071 - learning_rate: 0.0010
Epoch 4/20
680/680 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.9385 - loss: 0.1510 

2025-04-13 14:36:24.241274: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 12441856 bytes after encountering the first element of size 12441856 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


680/680 ━━━━━━━━━━━━━━━━━━━━ 43s 51ms/step - accuracy: 0.9385 - loss: 0.1510 - val_accuracy: 0.9101 - val_loss: 0.2399 - learning_rate: 0.0010
Epoch 5/20
680/680 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - accuracy: 0.9604 - loss: 0.0980 - val_accuracy: 0.9145 - val_loss: 0.2720 - learning_rate: 0.0010
Epoch 6/20
679/680 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9775 - loss: 0.0662 

2025-04-13 14:37:06.454517: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 12441856 bytes after encountering the first element of size 12441856 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


680/680 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - accuracy: 0.9775 - loss: 0.0661 - val_accuracy: 0.9050 - val_loss: 0.3307 - learning_rate: 0.0010


In [40]:
image = 'man.jpg'
image = tf.keras.utils.load_img(image,target_size=(img_size,img_size))
img_bat = tf.expand_dims(image,0)

In [58]:
def men_or_woman(value):
    if value > 0.5 :
        return 1
    else :
        return 0

In [42]:
predict = model.predict(img_bat)
predict_value = predict[0]
score = men_or_woman(predict_value)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step


In [43]:
print(f'THE GENDER OF PERSON IN IMAGE IS {data_category[score]} WITH ACCURACY OF {predict_value*100}')

THE GENDER OF PERSON IN IMAGE IS man WITH ACCURACY OF [0.04307611]


In [45]:
test_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [46]:
model.predict(test_dataset)

85/85 ━━━━━━━━━━━━━━━━━━━━ 14s 119ms/step


array([[0.00627946],
       [0.23200566],
       [0.00827636],
       ...,
       [0.0160919 ],
       [0.9996146 ],
       [0.53042984]], dtype=float32)

In [1]:
for i , (image,label) in enumerate(test_dataset.take(9)):
    plt.subplot(3,3,i+1)
    plt.imshow(image[0]/255)
    print(label.numpy()[0],model.predict(image)[0][0])
    plt.title(str(men_or_woman(label.numpy()[0]))+ ':' + str(men_or_woman(model.predict(image)[0][0])))
    plt.axis('off')

NameError: name 'test_dataset' is not defined

In [63]:
model.save('gender_classify.keras')